# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../weather/weather.csv")



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
weather_data = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)



### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_good_conditions = weather_data[weather_data['Humidity'] < 50]

weather_good_conditions = weather_good_conditions[weather_good_conditions['Wind Speed'] < 10]

weather_good_conditions = weather_good_conditions[weather_good_conditions['Temperature'] > 55]
weather_good_conditions = weather_good_conditions[weather_good_conditions['Temperature'] < 80]


weather_good_conditions

,City,City ID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
73,san quintin,3984997,MX,30.48,-115.95,61.57,40,7,6.91
81,new norfolk,2155415,AU,-42.78,147.06,68.76,29,100,3.00
85,santa barbara,5392967,US,34.73,-120.03,61.29,19,1,3.36
118,santiago del estero,3835869,AR,-27.80,-64.26,73.99,43,0,5.01
127,aguie,2448024,NE,13.51,7.78,66.54,25,0,8.77
144,san rafael,3836669,AR,-34.62,-68.33,71.22,33,0,7.87
146,guadalupe,4005492,MX,25.68,-100.25,63.55,38,5,3.36
176,nikki,2392505,BJ,9.94,3.21,70.61,33,0,3.49
231,la palma,5364022,US,33.85,-118.05,64.87,37,1,4.83
239,coronado,5339663,US,32.69,-117.18,63.86,42,20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = weather_good_conditions


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    params = {
    "location": f'{row.Latitude},{row.Longitude}', 
    'radius': '5000',
    #'circle':f'5000@{row.Latitude},{row.Longitude}',
    #"rankby": "distance",
    "type": "lodging",
    "key": g_key,
    'keyword' : 'hotel'
}
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}")
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'address'] = results[0]['vicinity']
        
            
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
 

Retrieving Results for Index 73
Closest hotel is Old Mill Hotel.
------------
Retrieving Results for Index 81
Closest hotel is The Woodbridge.
------------
Retrieving Results for Index 85
Missing field/result... skipping.
------------
Retrieving Results for Index 118
Closest hotel is Hotel Altos del Estero.
------------
Retrieving Results for Index 127
Missing field/result... skipping.
------------
Retrieving Results for Index 144
Closest hotel is Azalea Luxury Lodge.
------------
Retrieving Results for Index 146
Closest hotel is Fiesta Inn Monterrey Tecnológico.
------------
Retrieving Results for Index 176
Closest hotel is Hotel John.
------------
Retrieving Results for Index 231
Closest hotel is Knott's Berry Farm Hotel.
------------
Retrieving Results for Index 239
Closest hotel is Hilton San Diego Bayfront.
------------
Retrieving Results for Index 245
Closest hotel is Central Hotel.
------------
Retrieving Results for Index 269
Closest hotel is Stock Cafe At Real Inn.
-----------

In [8]:
hotel_df

,City,City ID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,address
73,san quintin,3984997,MX,30.48,-115.95,61.57,40,7,6.91,Old Mill Hotel,Ensenada
81,new norfolk,2155415,AU,-42.78,147.06,68.76,29,100,3.00,The Woodbridge,"6 Bridge St, New Norfolk"
85,santa barbara,5392967,US,34.73,-120.03,61.29,19,1,3.36,NaN,NaN
118,santiago del estero,3835869,AR,-27.80,-64.26,73.99,43,0,5.01,Hotel Altos del Estero,"Salta 40, Santiago del Estero"
127,aguie,2448024,NE,13.51,7.78,66.54,25,0,8.77,NaN,NaN
144,san rafael,3836669,AR,-34.62,-68.33,71.22,33,0,7.87,Azalea Luxury Lodge,"Las Virgenes 2882, San Rafael"
146,guadalupe,4005492,MX,25.68,-100.25,63.55,38,5,3.36,Fiesta Inn Monterrey Tecnológico,"Av. Eugenio Garza Sada 2411, Roma, Monterrey"
176,nikki,2392505,BJ,9.94,3.21,70.61,33,0,3.49,Hotel John,Nikki
231,la palma,5364022,US,33.85,-118.05,64.87,37,1,4.83,Knott's Berry Farm Hotel,"7675 Crescent Ave, Buena Park"
239,coronado,5339663,US,32.69,-117.18,63.86,42,20,6.93,Hilton San Diego Bayfront,"1 Park Blvd, San Diego"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, 
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))